In [ ]:
# Ячейка 1: Setup и загрузка моделей
import os
import subprocess
import glob
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth  # Только для Colab
from oauth2client.client import GoogleCredentials

# Функция клонирования репозитория, если папка ещё не существует
def clone_repo_if_not_exists(repo_url, dest_folder):
    if os.path.exists(dest_folder):
        print(f"[INFO] Репозиторий уже существует: {dest_folder}")
        return
    print(f"[INFO] Клонирование репозитория: {repo_url}")
    subprocess.run(['git', 'clone', repo_url, dest_folder], check=True)
    print(f"[INFO] Репозиторий успешно клонирован в: {dest_folder}")

# Функция для скачивания файла с Google Drive, если он ещё не скачан
def download_from_google_drive(file_id, file_dst):
    if os.path.exists(file_dst):
        print(f"[INFO] Файл уже существует: {file_dst}")
        return
    print(f"[INFO] Скачивание файла в: {file_dst}")
    downloaded = drive.CreateFile({'id': file_id})
    downloaded.FetchMetadata(fetch_all=True)
    downloaded.GetContentFile(file_dst)
    print(f"[INFO] Файл успешно скачан: {file_dst}")

# Аутентификация в Google Drive (требуется в Colab)
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

# Клонируем репозиторий HairMapper, если он ещё не существует
REPO_URL = "https://github.com/Lunatik-006/HairMapper.git"
REPO_FOLDER = "./HairMapper"
clone_repo_if_not_exists(REPO_URL, REPO_FOLDER)

# Переходим в папку репозитория
os.chdir(REPO_FOLDER)
print("[INFO] Текущая рабочая директория:", os.getcwd())

# Загрузка чекпоинтов моделей
checkpoints = {
    'StyleGAN2-ada-Generator.pth': {'url': '1EsGehuEdY4z4t21o2LgW2dSsyN3rxYLJ', 'dir': './ckpts'},
    'e4e_ffhq_encode.pt': {'url': '1cUv_reLE6k3604or78EranS7XzuVMWeO', 'dir': './ckpts'},
    'model_ir_se50.pth': {'url': '1GIMopzrt2GE_4PG-_YxmVqTQEiaqu5L6', 'dir': './ckpts'},
    'face_parsing.pth': {'url': '1IMsrkXA9NuCEy1ij8c8o6wCrAxkmjNPZ', 'dir': './ckpts'},
    'vgg16.pth': {'url': '1EPhkEP_1O7ZVk66aBeKoFqf3xiM4BHH8', 'dir': './ckpts'}
}
for ckpt_name, info in checkpoints.items():
    output_dir = info['dir']
    os.makedirs(output_dir, exist_ok=True)  # Создаем папку, если её нет
    output_path = os.path.join(output_dir, ckpt_name)
    download_from_google_drive(file_id=info['url'], file_dst=output_path)

# Загрузка чекпоинтов классификаторов (gender/hair)
classification_ckpt = [
    {'url': '1SSw6vd-25OGnLAE0kuA-_VHabxlsdLXL', 'dir': './classifier/gender_classification'},
    {'url': '1n14ckDcgiy7eu-e9XZhqQYb5025PjSpV', 'dir': './classifier/hair_classification'}
]
for clf in classification_ckpt:
    output_dir = clf['dir']
    os.makedirs(output_dir, exist_ok=True)
    output_path = os.path.join(output_dir, 'classification_model.pth')
    download_from_google_drive(file_id=clf['url'], file_dst=output_path)


[INFO] Клонирование репозитория: https://github.com/Lunatik-006/HairMapper.git
[INFO] Репозиторий успешно клонирован в: ./HairMapper
[INFO] Текущая рабочая директория: /content/HairMapper
[INFO] Скачивание файла в: ./ckpts/StyleGAN2-ada-Generator.pth
[INFO] Файл успешно скачан: ./ckpts/StyleGAN2-ada-Generator.pth
[INFO] Скачивание файла в: ./ckpts/e4e_ffhq_encode.pt
[INFO] Файл успешно скачан: ./ckpts/e4e_ffhq_encode.pt
[INFO] Скачивание файла в: ./ckpts/model_ir_se50.pth
[INFO] Файл успешно скачан: ./ckpts/model_ir_se50.pth
[INFO] Скачивание файла в: ./ckpts/face_parsing.pth
[INFO] Файл успешно скачан: ./ckpts/face_parsing.pth
[INFO] Скачивание файла в: ./ckpts/vgg16.pth
[INFO] Файл успешно скачан: ./ckpts/vgg16.pth
[INFO] Скачивание файла в: ./classifier/gender_classification/classification_model.pth
[INFO] Файл успешно скачан: ./classifier/gender_classification/classification_model.pth
[INFO] Скачивание файла в: ./classifier/hair_classification/classification_model.pth
[INFO] Файл у

In [ ]:
# Ячейка 2: Установка пакетов
!pip install torch===2.0.0+cu117 torchvision===0.15.0+cu117 torchaudio===2.0.0+cu117 -f https://download.pytorch.org/whl/torch_stable.html
!pip install -r requirements.txt
!pip install "numpy<2.0"
!wget https://github.com/ninja-build/ninja/releases/download/v1.8.2/ninja-linux.zip
!sudo unzip ninja-linux.zip -d /usr/local/bin/
!sudo update-alternatives --install /usr/bin/ninja ninja /usr/local/bin/ninja 1 --force


Looking in links: https://download.pytorch.org/whl/torch_stable.html
  Using cached tqdm-4.60.0-py2.py3-none-any.whl.metadata (57 kB)
  Using cached requests-2.25.1-py2.py3-none-any.whl.metadata (4.2 kB)
  Using cached matplotlib-3.4.1.tar.gz (37.3 MB)
  Preparing metadata (setup.py) ... done
  Using cached PyYAML-5.4.1.tar.gz (175 kB)
  Installing build dependencies ... done
  error: subprocess-exited-with-error
  
  × Getting requirements to build wheel did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Getting requirements to build wheel ... error
error: subprocess-exited-with-error

× Getting requirements to build wheel did not run successfully.
│ exit code: 1
╰─> See above for output.

note: This error originates from a subprocess, and is likely not a problem with pip.
--2025-04-07 23:01:54--  https://github.com/ninja-build/ninja/releases/download/v1.8.2/ninja-linux.zip


In [ ]:
from argparse import Namespace
import sys
import torch
import torchvision.transforms as transforms
import numpy as np
import PIL.Image
from PIL import ImageFile
import glob
import os
import argparse


# Здесь загружается модель mapper из папки ./mapper/checkpoints/final
mapper_url = 'https://drive.google.com/file/d/1F3oujXbvalqEOixcAkIyURuY512nmroe'
mapper_id = mapper_url.replace('https://drive.google.com/file/d/', '').split('/')[0]
mapper_output_dir = './mapper/checkpoints/final'
os.makedirs(mapper_output_dir, exist_ok=True)
mapper_output_path = os.path.join(mapper_output_dir, 'best_model.pt')
download_from_google_drive(file_id=mapper_id, file_dst=mapper_output_path)

# Импорт необходимых классов для mapper и генератора
# (Убедитесь, что пути импорта соответствуют структуре репозитория HairMapper)
from mapper.networks.level_mapper import LevelMapper
from styleGAN2_ada_model.stylegan2_ada_generator import StyleGAN2adaGenerator
from classifier.src.feature_extractor.hair_mask_extractor import get_hair_mask, get_parsingNet

# Инициализация генератора (StyleGAN2-ada)
model_name = 'stylegan2_ada'
print("[INFO] Инициализация генератора.")
model = StyleGAN2adaGenerator(model_name, logger=None, truncation_psi=1.0)

# Инициализация mapper
mapper = LevelMapper(input_dim=512).eval().cuda()
mapper_ckpt = torch.load(mapper_output_path, map_location='cpu')
alpha = float(mapper_ckpt['alpha']) * 1.2
mapper.load_state_dict(mapper_ckpt['state_dict'], strict=True)

# Параметры для генератора
latent_space_type = 'wp'
kwargs = {'latent_space_type': latent_space_type}

# Загрузка парсинговой сети для извлечения маски волос /content/HairMapper/ckpts/face_parsing.pth
parsingNet = get_parsingNet(save_pth='/content/HairMapper/ckpts/face_parsing.pth')

# Тестовое изображение загружается в папку ./test_data/origin (для отладки)
test_img_name = 'test_img.png'
test_img_id = '1Ju5jLtNCALHJ2crJkMr00UP_ZUQzQBRs'
test_img_dir = './test_data/origin'
os.makedirs(test_img_dir, exist_ok=True)
test_img_path = os.path.join(test_img_dir, test_img_name)
download_from_google_drive(file_id=test_img_id, file_dst=test_img_path)


[INFO] Файл уже существует: ./mapper/checkpoints/final/best_model.pt
[INFO] Инициализация генератора.
Loading pytorch model from `/content/HairMapper/ckpts/StyleGAN2-ada-Generator.pth`.
load face_parsing model from:  /content/HairMapper/ckpts/face_parsing.pth
[INFO] Скачивание файла в: ./test_data/origin/test_img.png
[INFO] Файл успешно скачан: ./test_data/origin/test_img.png


In [ ]:
# Ячейка 4: Переход в папку encoder4editing и загрузка модели pSp (энкодер e4e)
os.chdir('/content/HairMapper/encoder4editing')
print("[INFO] Текущая директория (encoder4editing):", os.getcwd())

import PIL._util
if not hasattr(PIL._util, 'is_directory'):
    import os
    PIL._util.is_directory = lambda path: os.path.isdir(path)


from argparse import Namespace
import sys
import torch
import torchvision.transforms as transforms
import numpy as np
import PIL.Image
from PIL import ImageFile
import glob
import argparse

sys.path.append(".")
sys.path.append("..")
ImageFile.LOAD_TRUNCATED_IMAGES = True

from models.psp import pSp  # Импорт модели pSp

# Трансформации для входного изображения (из папки с исходными изображениями)
img_transforms = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor(),
    transforms.Normalize([0.5,0.5,0.5], [0.5,0.5,0.5])
])

# Загрузка чекпоинта энкодера e4e из папки ./ckpts
model_path = "../ckpts/e4e_ffhq_encode.pt"
ckpt = torch.load(model_path, map_location='cpu')
opts = ckpt['opts']
opts['checkpoint_path'] = model_path
opts = Namespace(**opts)
net = pSp(opts)
net.eval()
net.cuda()
print("[INFO] Модель pSp (энкодер e4e) загружена.")


[INFO] Текущая директория (encoder4editing): /content/HairMapper/encoder4editing
Loading e4e over the pSp framework from checkpoint: ../ckpts/e4e_ffhq_encode.pt
[INFO] Модель pSp (энкодер e4e) загружена.


In [ ]:
#############################
# Ячейка 5: Обработка изображений из папки на Google Drive с входными данными
#############################
import zipfile
import shutil
from google.colab import files

# Укажите ID входной папки на Google Drive с исходными изображениями
# (например, ссылка вида https://drive.google.com/drive/folders/XXX, где XXX - это ID)
input_folder_id = '15QuG_Iu8JAmVOJ-9HEJzBrk80NujHbqb'  # Замените на реальный ID входной папки

# Локальные папки для временного хранения входных и выходных изображений
temp_input_folder = './temp_input'
os.makedirs(temp_input_folder, exist_ok=True)
output_folder = './temp_output'
os.makedirs(output_folder, exist_ok=True)

# Получение списка файлов из входной папки по MIME-типу (png и jpg)
query = f"'{input_folder_id}' in parents and (mimeType='image/png')"
input_file_list = drive.ListFile({'q': query}).GetList()
total_files = len(input_file_list)
print(f"[INFO] Найдено {total_files} входных изображений.")

from tqdm import tqdm
import cv2
import numpy as np
import PIL.Image

pbar = tqdm(total=total_files)
for f in input_file_list:
    pbar.update(1)
    file_title = f['title']
    # Скачиваем файл во временную папку
    local_input_path = os.path.join(temp_input_folder, file_title)
    f.GetContentFile(local_input_path)

    # Открываем изображение с помощью PIL и применяем трансформации
    try:
        input_img = PIL.Image.open(local_input_path).convert('RGB')
    except Exception as e:
        print(f"[ERROR] Не удалось открыть {file_title}: {e}")
        continue
    transformed_image = img_transforms(input_img)

    # Получаем латентное представление с помощью модели pSp
    with torch.no_grad():
        latents = net(transformed_image.unsqueeze(0).cuda().float(), randomize_noise=False, return_latents=True)
        latent = latents[0].cpu().numpy()
        latent = np.reshape(latent, (1, 18, 512))

    # Применяем mapper для корректировки латентного кода (удаление волос)
    mapper_input = latent.copy()
    mapper_input_tensor = torch.from_numpy(mapper_input).cuda().float()
    edited_latent_codes = latent.copy()
    edited_latent_codes[:, :8, :] += alpha * mapper(mapper_input_tensor).to('cpu').detach().numpy()

    # Генерация нового изображения с помощью генератора StyleGAN2-ada
    outputs = model.easy_style_mixing(latent_codes=edited_latent_codes,
                                      style_range=range(7, 18),
                                      style_codes=latent,
                                      mix_ratio=0.8,
                                      **kwargs)
    edited_img = outputs['image'][0][:, :, ::-1]  # Преобразование из BGR в RGB

    # Получаем исходное изображение для получения маски (считываем локально скачанный файл)
    origin_img = cv2.imread(local_input_path)

    # Извлекаем маску волос и применяем seamlessClone
    hair_mask = get_hair_mask(img_path=origin_img, net=parsingNet, include_hat=True, include_ear=True)
    mask_dilate = cv2.dilate(hair_mask, kernel=np.ones((50, 50), np.uint8))
    mask_dilate_blur = cv2.blur(mask_dilate, ksize=(30, 30))
    mask_dilate_blur = (hair_mask + (255 - hair_mask)/255 * mask_dilate_blur).astype(np.uint8)
    face_mask = 255 - mask_dilate_blur
    face_mask = cv2.resize(face_mask, (origin_img.shape[1], origin_img.shape[0]))
    idx = np.where(face_mask > 0)
    cy = (np.min(idx[0]) + np.max(idx[0])) // 2
    cx = (np.min(idx[1]) + np.max(idx[1])) // 2
    center = (cx, cy)

    # Применяем seamlessClone для объединения изображений
    result_img = cv2.seamlessClone(origin_img, edited_img, face_mask[:, :, 0], center, cv2.NORMAL_CLONE)

    # Сохраняем обработанное изображение в папку output_folder с расширением .png
    output_filename = os.path.splitext(file_title)[0] + '.png'
    local_output_path = os.path.join(output_folder, output_filename)
    cv2.imwrite(local_output_path, result_img)
pbar.close()

print("[INFO] Обработка завершена. Обработанные изображения сохранены в:", output_folder)

# Создаем ZIP-архив из папки с обработанными изображениями
zip_filename = "processed_images.zip"
!zip -r {zip_filename} {output_folder}
print(f"[INFO] Архив {zip_filename} создан.")

# Автоматическое скачивание ZIP-архива на ПК
files.download(zip_filename)

[INFO] Найдено 9897 входных изображений.


  0%|          | 0/9897 [00:00<?, ?it/s]

Setting up PyTorch plugin "bias_act_plugin"...


Using /root/.cache/torch_extensions/py311_cu124 as PyTorch extensions root...
Creating extension directory /root/.cache/torch_extensions/py311_cu124/bias_act_plugin...


RuntimeError: Ninja is required to load C++ extensions

In [10]:
import os
import math
import zipfile
from google.colab import files
from google.colab import drive
drive.mount('/content/drive')
import shutil



# Путь к папке с обработанными изображениями
output_folder = './temp_output'

# Получаем список файлов в папке
all_files = sorted(os.listdir(output_folder))
total_files = len(all_files)
print(f"[INFO] Всего найдено {total_files} файлов.")

# Определяем размер батча (количество изображений в одном архиве)
batch_size = 1000
num_batches = math.ceil(total_files / batch_size)
print(f"[INFO] Будет создано {num_batches} архив(ов), по {batch_size} изображений (последний архив может быть меньше).")

# Создаём архивы по батчам и инициируем скачивание каждого архива
for batch in range(num_batches):
    start_idx = batch * batch_size
    end_idx = min((batch + 1) * batch_size, total_files)
    batch_files = all_files[start_idx:end_idx]
    zip_filename = f"processed_images_batch_{batch+1}.zip"

    with zipfile.ZipFile(zip_filename, 'w', compression=zipfile.ZIP_DEFLATED) as zipf:
        for file in batch_files:
            file_path = os.path.join(output_folder, file)
            # Записываем файл в архив с относительным путём (без полного пути)
            zipf.write(file_path, arcname=file)

    print(f"[INFO] Архив {zip_filename} создан с файлами {start_idx+1} – {end_idx}.")
    # Скачиваем архив
    #files.download(zip_filename)
    shutil.move(zip_filename, f'/content/drive/MyDrive/{zip_filename}')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
[INFO] Всего найдено 9803 файлов.
[INFO] Будет создано 10 архив(ов), по 1000 изображений (последний архив может быть меньше).
[INFO] Архив processed_images_batch_1.zip создан с файлами 1 – 1000.
[INFO] Архив processed_images_batch_2.zip создан с файлами 1001 – 2000.
[INFO] Архив processed_images_batch_3.zip создан с файлами 2001 – 3000.
[INFO] Архив processed_images_batch_4.zip создан с файлами 3001 – 4000.
[INFO] Архив processed_images_batch_5.zip создан с файлами 4001 – 5000.
[INFO] Архив processed_images_batch_6.zip создан с файлами 5001 – 6000.
[INFO] Архив processed_images_batch_7.zip создан с файлами 6001 – 7000.
[INFO] Архив processed_images_batch_8.zip создан с файлами 7001 – 8000.
[INFO] Архив processed_images_batch_9.zip создан с файлами 8001 – 9000.
[INFO] Архив processed_images_batch_10.zip создан с файлами 9001 – 9803.
